# Imports

In [30]:
import argparse
import time
import os

import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader

from mscn.util import *
from mscn.data import get_train_datasets, load_data, make_dataset
from mscn.model import SetConv

# Functions

In [32]:
def unnormalize_torch(vals, min_val, max_val):
    vals = (vals * (max_val - min_val)) + min_val
    return torch.exp(vals)


def qerror_loss(preds, targets, min_val, max_val):
    qerror = []
    preds = unnormalize_torch(preds, min_val, max_val)
    targets = unnormalize_torch(targets, min_val, max_val)

    for i in range(len(targets)):
        if (preds[i] > targets[i]).cpu().data.numpy()[0]:
            qerror.append(preds[i] / targets[i])
        else:
            qerror.append(targets[i] / preds[i])
    return torch.mean(torch.cat(qerror))


def predict(model, data_loader, cuda):
    preds = []
    t_total = 0.

    model.eval()
    for batch_idx, data_batch in enumerate(data_loader):

        samples, predicates, joins, targets, sample_masks, predicate_masks, join_masks = data_batch

        if cuda:
            samples, predicates, joins, targets = samples.cuda(), predicates.cuda(), joins.cuda(), targets.cuda()
            sample_masks, predicate_masks, join_masks = sample_masks.cuda(), predicate_masks.cuda(), join_masks.cuda()
        samples, predicates, joins, targets = Variable(samples), Variable(predicates), Variable(joins), Variable(
            targets)
        sample_masks, predicate_masks, join_masks = Variable(sample_masks), Variable(predicate_masks), Variable(
            join_masks)

        t = time.time()
        outputs = model(samples, predicates, joins, sample_masks, predicate_masks, join_masks)
        t_total += time.time() - t

        for i in range(outputs.data.shape[0]):
            preds.append(outputs.data[i])

    return preds, t_total


def print_qerror(preds_unnorm, labels_unnorm):
    qerror = []
    for i in range(len(preds_unnorm)):
        # SQ: preds_unnorm[i] is an array whereas labels_unnorm[i] is a scaler (int64)
        # It was causing an error, so I changed the following code to unpack the scaler value
        # from each array element inside preds_unnorm
        if preds_unnorm[i][0] > float(labels_unnorm[i]):
            qerror.append(preds_unnorm[i][0] / float(labels_unnorm[i]))
        else:
            qerror.append(float(labels_unnorm[i]) / float(preds_unnorm[i][0]))

    print("Median: {}".format(np.median(qerror)))
    print("90th percentile: {}".format(np.percentile(qerror, 90)))
    print("95th percentile: {}".format(np.percentile(qerror, 95)))
    print("99th percentile: {}".format(np.percentile(qerror, 99)))
    print("Max: {}".format(np.max(qerror)))
    print("Mean: {}".format(np.mean(qerror)))

In [33]:
# print(args.testset)
# train_and_predict(args.testset, args.queries, args.epochs, args.batch, args.hid, args.cuda)
#workload_name = 'job-light'
workload_name = 'test_tpcds'
num_queries = 99
num_epochs = 10
batch_size = 1024
hid_units = 256
cuda = False

In [22]:
# Load training and validation data
num_materialized_samples = 1000
dicts, column_min_max_vals, min_val, max_val, labels_train, labels_test, max_num_joins, max_num_predicates, train_data, test_data = get_train_datasets(num_queries, num_materialized_samples)
table2vec, column2vec, op2vec, join2vec = dicts


Loaded queries
Loaded bitmaps
min log(label): 0.0
max log(label): 19.94772801931604
Number of training samples: 89
Number of validation samples: 10
Created TensorDataset for training data
Created TensorDataset for validation data


In [23]:
dicts

[{'cast_info ci': array([1., 0., 0., 0., 0., 0.], dtype=float32),
  'movie_companies mc': array([0., 1., 0., 0., 0., 0.], dtype=float32),
  'movie_info mi': array([0., 0., 1., 0., 0., 0.], dtype=float32),
  'movie_info_idx mi_idx': array([0., 0., 0., 1., 0., 0.], dtype=float32),
  'movie_keyword mk': array([0., 0., 0., 0., 1., 0.], dtype=float32),
  'title t': array([0., 0., 0., 0., 0., 1.], dtype=float32)},
 {'ci.person_id': array([1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
  'ci.role_id': array([0., 1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
  'mc.company_id': array([0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32),
  'mc.company_type_id': array([0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32),
  'mi.info_type_id': array([0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32),
  'mi_idx.info_type_id': array([0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32),
  'mk.keyword_id': array([0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32),
  't.kind_id': array([0., 

In [28]:

# Train model
sample_feats = len(table2vec) + num_materialized_samples
predicate_feats = len(column2vec) + len(op2vec) + 1
join_feats = len(join2vec)

model = SetConv(sample_feats, predicate_feats, join_feats, hid_units)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


train_data_loader = DataLoader(train_data, batch_size=batch_size)
test_data_loader = DataLoader(test_data, batch_size=batch_size)

model.train()

for epoch in range(num_epochs):
    loss_total = 0.

    for batch_idx, data_batch in enumerate(train_data_loader):
        samples, predicates, joins, targets, sample_masks, predicate_masks, join_masks = data_batch

        samples, predicates, joins, targets = Variable(samples), Variable(predicates), Variable(joins), Variable(targets)
        sample_masks, predicate_masks, join_masks = Variable(sample_masks), Variable(predicate_masks), Variable(join_masks)

        optimizer.zero_grad()
        outputs = model(samples, predicates, joins, sample_masks, predicate_masks, join_masks)
        loss = qerror_loss(outputs, targets.float(), min_val, max_val)
        loss_total += loss.item()
        loss.backward()
        optimizer.step()

    print("Epoch {}, loss: {}".format(epoch, loss_total / len(train_data_loader)))

# Get final training and validation set predictions
preds_train, t_total = predict(model, train_data_loader, cuda)
print("Prediction time per training sample: {}".format(t_total / len(labels_train) * 1000))

preds_test, t_total = predict(model, test_data_loader, cuda)
print("Prediction time per validation sample: {}".format(t_total / len(labels_test) * 1000))

# Unnormalize
preds_train_unnorm = unnormalize_labels(preds_train, min_val, max_val)
labels_train_unnorm = unnormalize_labels(labels_train, min_val, max_val)

preds_test_unnorm = unnormalize_labels(preds_test, min_val, max_val)
labels_test_unnorm = unnormalize_labels(labels_test, min_val, max_val)

# Print metrics
print("\nQ-Error training set:")
print_qerror(preds_train_unnorm, labels_train_unnorm)

print("\nQ-Error validation set:")
print_qerror(preds_test_unnorm, labels_test_unnorm)
print("")

# Load test data
file_name = "workloads/" + workload_name
joins, predicates, tables, samples, label = load_data(file_name, num_materialized_samples)

# Get feature encoding and proper normalization
samples_test = encode_samples(tables, samples, table2vec)
predicates_test, joins_test = encode_data(predicates, joins, column_min_max_vals, column2vec, op2vec, join2vec)
labels_test, _, _ = normalize_labels(label, min_val, max_val)

print("Number of test samples: {}".format(len(labels_test)))

max_num_predicates = max([len(p) for p in predicates_test])
max_num_joins = max([len(j) for j in joins_test])

# Get test set predictions
test_data = make_dataset(samples_test, predicates_test, joins_test, labels_test, max_num_joins, max_num_predicates)
test_data_loader = DataLoader(test_data, batch_size=batch_size)

preds_test, t_total = predict(model, test_data_loader, cuda)
print("Prediction time per test sample: {}".format(t_total / len(labels_test) * 1000))

# Unnormalize
preds_test_unnorm = unnormalize_labels(preds_test, min_val, max_val)

# Print metrics
print("\nQ-Error " + workload_name + ":")
print_qerror(preds_test_unnorm, label)

# Write predictions
file_name = "results/predictions_" + workload_name + ".csv"
os.makedirs(os.path.dirname(file_name), exist_ok=True)
with open(file_name, "w") as f:
    for i in range(len(preds_test_unnorm)):
        f.write(str(preds_test_unnorm[i]) + "," + label[i] + "\n")


Epoch 0, loss: 736.8335571289062
Epoch 1, loss: 869.8615112304688
Epoch 2, loss: 401.494384765625
Epoch 3, loss: 402.35174560546875
Epoch 4, loss: 361.444091796875
Epoch 5, loss: 287.44708251953125
Epoch 6, loss: 261.7984619140625
Epoch 7, loss: 227.74668884277344
Epoch 8, loss: 176.35147094726562
Epoch 9, loss: 151.7735137939453
Prediction time per training sample: 0.01826982819632198
Prediction time per validation sample: 0.0553131103515625

Q-Error training set:
Median: 23.38071604863853
90th percentile: 237.96936131777872
95th percentile: 354.34142259421674
99th percentile: 1343.1812062252259
Max: 4241.5
Mean: 130.94419836310914

Q-Error validation set:
Median: 19.54470109243407
90th percentile: 128.16981099862375
95th percentile: 492.4071563687199
99th percentile: 783.7970326647983
Max: 856.6445017388176
Mean: 101.817271944403



ValueError: invalid literal for int() with base 10: '32252.0'